# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [2]:
# 打開瀏覽器

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd

browser = webdriver.Chrome(executable_path='../chromedriver')

browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

# 模擬使用者操作行為，選擇/點擊

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')

browser.find_element_by_id('ctl05_btnQuery').click()


In [3]:
# 取得資料，丟到 BeautifulSoup 解析

html_source = browser.page_source

soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')

trs = table.find_all('tr')
category_name = ''
date_data = {}
d = {}
idx = 1

for  tr in trs[1:9]:
    category_td = tr.find('td', class_='no-alt')
    category_name = category_td.text if category_td else category_name
    date = tr.find_all('td', class_=None)[0].text
    data = tr.find_all('td', class_=None)[1].text
    date_data[date] = data
    if idx == 8:
        date = tr.find_all('td', class_=None)[0].text
        data = tr.find_all('td', class_=None)[1].text
        date_data[date] = data
        d[category_name] = date_data

    idx += 1  

print(pd.DataFrame(d))

          SO2
2018/01  1.80
2018/02  1.90
2018/03  2.20
2018/04  2.30
2018/05  3.10
2018/06  2.70
2018/07  2.20
2018/08  2.40


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [4]:
trs = table.find_all('tr')
category_name = ''
date_data = {}
d = {}
idx = 1

for  tr in trs[1:9]+trs[13:21]:
    category_td = tr.find('td', class_='no-alt')
    category_name = category_td.text if category_td else category_name
    date = tr.find_all('td', class_=None)[0].text
    data = tr.find_all('td', class_=None)[1].text
    date_data[date] = data
    if idx == 8 or idx == 16:
        date = tr.find_all('td', class_=None)[0].text
        data = tr.find_all('td', class_=None)[1].text
        date_data[date] = data
        d[category_name] = date_data
        date_data = {}
    idx += 1  

print(pd.DataFrame(d))

          SO2    CO
2018/01  1.80  0.34
2018/02  1.90  0.44
2018/03  2.20  0.40
2018/04  2.30  0.38
2018/05  3.10  0.34
2018/06  2.70  0.29
2018/07  2.20  0.21
2018/08  2.40  0.30


In [5]:
# 關閉瀏覽器
browser.quit();